In [1]:
import numpy as np
import pickle
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import corpus_bleu

Using TensorFlow backend.


In [2]:
with open("../saved_models and files/word_to_idx.pkl","rb") as w2i:
    word_to_idx=pickle.load(w2i)
with open("../saved_models and files/idx_to_word.pkl","rb") as i2w:
    idx_to_word=pickle.load(i2w)
with open("../saved_models and files/test_featured_image.pkl","rb") as tfi:
    test_featured_image=pickle.load(tfi)
with open("../saved_models and files/test_description.pkl","rb") as td:
    test_description=pickle.load(td)
with open("../saved_models and files/train_featured_image.pkl","rb") as tfi:
    train_featured_image=pickle.load(tfi)

In [13]:
model = load_model('../save_model_weight4/model_12.h5')

C:\Users\yash\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [14]:
max_len=35
def predict_caption(photo):
    in_text="startseq"
    for i in range(max_len):
        sequence = [word_to_idx[w] for w in in_text.split() if w in word_to_idx]
        sequence = pad_sequences([sequence],maxlen=max_len,padding='post')
        
        y_pred = model.predict([photo,sequence])
        y_pred = y_pred.argmax()
        word = idx_to_word[y_pred]
        in_text+=(' '+word)

        if word == 'endseq':
            break
    final_caption = in_text.split()[1:-1]
    final_caption = ' '.join(final_caption)
    return final_caption

In [15]:
def evaluate_model():
    actual, predicted = list(), list()
# step over the whole set
    for key, desc_list in test_description.items():
        photo_2048 = test_featured_image[key].reshape((1,2048))
    # generate description
        yhat = predict_caption(photo_2048)
    # store actual and predicted
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [16]:
evaluate_model()

BLEU-1: 0.420399
BLEU-2: 0.222130
BLEU-3: 0.142831
BLEU-4: 0.060948


<img src="score.png">

In [ ]:
plt.style.use('seaborn')
for i in range(20):
    idx = np.random.randint(0,5000)
    all_img_names = list(test_featured_image.keys())
    img_name = all_img_names[idx]
    photo_2048 = test_featured_image[img_name].reshape((1,2048))

    i=plt.imread("../flickr30k_images/flickr30k_images/"+img_name)

    caption = predict_caption(photo_2048)

    plt.title(caption)
    plt.imshow(i)
    plt.axis('off')
    plt.show()